In [3]:
import random
import os
import cv2
import numpy as np
from pathlib import Path
from PIL import Image
import torch
from tqdm import tqdm
import xml.etree.ElementTree as ET

In [2]:
example_background = torch.load('/home/gridsan/manderson/ovdsat/run/coop_prototypes/boxes/no_bg/dior_N5-1/prototypes_remoteclip-14.pt')

In [10]:
example_background['label_names'][:10]

['bg_class_1',
 'bg_class_2',
 'bg_class_3',
 'bg_class_4',
 'bg_class_5',
 'bg_class_6',
 'bg_class_7',
 'bg_class_8',
 'bg_class_9',
 'bg_class_10']

In [7]:
example_background['prototypes'].shape

torch.Size([200, 1024])

## Generate crops for few shot training sets
* Run coco_to_seg first to create images with masks (if not done already!)
* Crop images for each N shot and M split and save to separate folder
* Extract 10 random background crops per image and save in separate folder

In [7]:
def apply_rectangular_mask_and_crop(image: Image.Image, mask: Image.Image) -> Image.Image:
    """Crops the image to the rectangular mask bounding box only (no masking)."""
    mask_array = np.array(mask.convert("L"))

    # Get non-zero mask coordinates
    ys, xs = np.where(mask_array > 0)
    if xs.size == 0 or ys.size == 0:
        return Image.new("RGB", (1, 1))  # Blank image if no mask

    x_min, x_max = xs.min(), xs.max()
    y_min, y_max = ys.min(), ys.max()

    # Crop the image directly to the mask's bounding rectangle
    return image.crop((x_min, y_min, x_max + 1, y_max + 1))

def process_directory(input_dir: str, output_dir: str):
    input_path = Path(input_dir)
    output_path = Path(output_dir)

    for root, _, files in os.walk(input_path):
        for file in files:
            if file.endswith(".jpg") and not file.endswith(".mask.jpg"):
                image_file = Path(root) / file
                mask_file = image_file.with_name(f"{image_file.stem}.mask.jpg")

                if not mask_file.exists():
                    print(f"[!] Mask not found for {image_file.name}, skipping.")
                    continue

                try:
                    image = Image.open(image_file)
                    mask = Image.open(mask_file)
                    masked_cropped = apply_rectangular_mask_and_crop(image, mask)

                    # Determine output path
                    relative_path = image_file.relative_to(input_path)
                    output_image_path = output_path / relative_path
                    output_image_path.parent.mkdir(parents=True, exist_ok=True)

                    masked_cropped.save(output_image_path)
                    print(f"Saved: {output_image_path}")
                except Exception as e:
                    print(f"Error processing {image_file}: {e}")

In [13]:
# For few-shot train

for N in [5, 10, 30]:
    for M in [1, 2, 3, 4, 5]:
        print(f'================ Processing train N={N}, M={M} ================')
        input_root = f'/home/gridsan/manderson/ovdsat/data/init_data/dior_N{N}-{M}'
        output_root = f'/home/gridsan/manderson/ovdsat/data/cropped_data/dior_N{N}-{M}'
        process_directory(input_root, output_root)
        print()

================ Processing N=5, M=1 ================
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior_N5-1/ship/11728.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior_N5-1/ship/03398.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior_N5-1/ship/01572.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior_N5-1/ship/12329.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior_N5-1/ship/22752.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior_N5-1/harbor/00364.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior_N5-1/harbor/15432.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior_N5-1/harbor/12862.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior_N5-1/harbor/08505.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior_N5-1/Expressway-toll-station/14865.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior_N5-1/Expressway-toll-station/03962.jpg
Saved: /h

In [26]:
# For val

for M in [1, 2, 3, 4, 5]:
    print(f'================ Processing val M={M} ================')
    input_root = f'/home/gridsan/manderson/ovdsat/data/init_data/dior_val-{M}'
    output_root = f'/home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-{M}'
    process_directory(input_root, output_root)
    print()

================ Processing val M=3 ================
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-3/ship/05669.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-3/ship/10468.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-3/ship/07966.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-3/ship/13722.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-3/ship/07678.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-3/ship/05575.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-3/ship/18789.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-3/ship/04490.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-3/ship/12626.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-3/ship/06429.jpg
Saved: /home/gridsan/manderson/ovdsat

In [ ]:
# For test

for M in [1]:#, 2, 3, 4, 5]:
    print(f'================ Processing test M={M} ================')
    input_root = f'/home/gridsan/manderson/ovdsat/data/init_data/dior_test-{M}'
    output_root = f'/home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-{M}'
    process_directory(input_root, output_root)
    print()

================ Processing test M=1 ================
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/12175.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/17292.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/06433.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/00995.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/20232.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/06848.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/04461.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/14030.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/19324.jpg
Saved: /home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-1/ship/04535.jpg
Saved: /home/gri

Extract 10 background crops per sample

In [27]:
def generate_background_boxes_from_mask(mask_array, num_b, min_size=30, max_size=200, max_iter=100):
    """
    Generate boxes that do NOT intersect with non-zero mask areas.
    """
    h, w = mask_array.shape
    boxes = []

    for _ in range(num_b):
        valid_box = False
        count = 0
        while not valid_box and count < max_iter:
            count += 1
            width = random.randint(min_size, min(max_size, w))
            height = random.randint(min_size, min(max_size, h))
            x = random.randint(0, w - width)
            y = random.randint(0, h - height)
            x2 = x + width
            y2 = y + height

            # Check if the box intersects with foreground mask
            if np.any(mask_array[y:y2, x:x2] > 0):
                continue  # intersecting: skip
            boxes.append((x, y, x2, y2))
            valid_box = True

    return boxes

def generate_background_crops_from_masks(input_dir, output_root, num_b=10, min_size=30, max_size=200):
    input_path = Path(input_dir)
    background_dir = Path(output_root) / "background"
    background_dir.mkdir(parents=True, exist_ok=True)

    for root, _, files in os.walk(input_path):
        for file in tqdm(files):
            if file.endswith(".jpg") and not file.endswith(".mask.jpg"):
                image_file = Path(root) / file
                mask_file = image_file.with_name(f"{image_file.stem}.mask.jpg")

                if not mask_file.exists():
                    print(f"[!] No mask found for {image_file.name}, skipping.")
                    continue

                try:
                    image = Image.open(image_file).convert("RGB")
                    mask = Image.open(mask_file).convert("L")
                except Exception as e:
                    print(f"[✗] Error loading {image_file.name}: {e}")
                    continue

                mask_array = np.array(mask)
                boxes = generate_background_boxes_from_mask(mask_array, num_b, min_size, max_size)

                for i, (x1, y1, x2, y2) in enumerate(boxes):
                    crop = image.crop((x1, y1, x2, y2))
                    crop_filename = f"{image_file.stem}_background{i}.jpg"
                    crop.save(background_dir / crop_filename)
                    #print(f"[✓] Saved: {background_dir / crop_filename}")

In [20]:
# For train

for N in [5, 10, 30]:
    for M in [1, 2, 3, 4, 5]:
        print(f'================ Processing train N={N}, M={M} ================')
        input_root = f'/home/gridsan/manderson/ovdsat/data/init_data/dior_N{N}-{M}'
        output_root = f'/home/gridsan/manderson/ovdsat/data/cropped_data/dior_N{N}-{M}'
        generate_background_crops_from_masks(input_root, output_root)
        print()

================ Processing N=10, M=1 ================


0it [00:00, ?it/s]
100%|██████████| 2/2 [00:00<00:00, 3135.93it/s]


[!] No mask found for 19192.mask-checkpoint.jpg, skipping.
[!] No mask found for 19192-checkpoint.jpg, skipping.


100%|██████████| 20/20 [00:06<00:00,  3.03it/s]



================ Processing N=10, M=2 ================


...

0it [00:00, ?it/s]
100%|██████████| 60/60 [00:07<00:00,  8.36it/s]



================ Processing N=30, M=4 ================


0it [00:00, ?it/s]
100%|██████████| 60/60 [00:06<00:00,  8.68it/s]



================ Processing N=30, M=5 ================


0it [00:00, ?it/s]
100%|██████████| 60/60 [00:05<00:00, 11.41it/s]

In [28]:
# For val

for M in [1, 2, 3, 4, 5]:
    print(f'================ Processing val M={M} ================')
    input_root = f'/home/gridsan/manderson/ovdsat/data/init_data/dior_val-{M}'
    output_root = f'/home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior_val-{M}'
    generate_background_crops_from_masks(input_root, output_root)
    print()

================ Processing val M=1 ================


0it [00:00, ?it/s]
100%|██████████| 20/20 [00:00<00:00, 23.36it/s]



================ Processing val M=2 ================


0it [00:00, ?it/s]
100%|██████████| 20/20 [00:00<00:00, 32.34it/s]



================ Processing val M=3 ================


0it [00:00, ?it/s]
100%|██████████| 20/20 [00:00<00:00, 30.17it/s]



================ Processing val M=4 ================


0it [00:00, ?it/s]
100%|██████████| 20/20 [00:00<00:00, 32.37it/s]



================ Processing val M=5 ================


0it [00:00, ?it/s]
100%|██████████| 20/20 [00:00<00:00, 35.16it/s]

In [30]:
# For test

for M in [1, 2, 3, 4, 5]:
    print(f'================ Processing test M={M} ================')
    input_root = f'/home/gridsan/manderson/ovdsat/data/init_data/dior_test-{M}'
    output_root = f'/home/gridsan/manderson/ovdsat/data/cropped_data/dior/test/dior_test-{M}'
    generate_background_crops_from_masks(input_root, output_root, num_b=1) # just get one crop per image
    print()

================ Processing test M=1 ================


0it [00:00, ?it/s]
100%|██████████| 200/200 [00:01<00:00, 118.14it/s]



================ Processing test M=2 ================


0it [00:00, ?it/s]
100%|██████████| 200/200 [00:02<00:00, 99.12it/s] 



================ Processing test M=3 ================


0it [00:00, ?it/s]
100%|██████████| 200/200 [00:01<00:00, 107.28it/s]



================ Processing test M=4 ================


0it [00:00, ?it/s]
100%|██████████| 200/200 [00:02<00:00, 92.16it/s] 



================ Processing test M=5 ================


0it [00:00, ?it/s]
100%|██████████| 200/200 [00:02<00:00, 90.98it/s] 

## Create images + masks (init_data)

In [5]:
import os
import json
import cv2
import numpy as np
from PIL import Image
from pycocotools.coco import COCO

def coco_to_seg(annotation_file, image_directory, save_path, subset_size=None):
    '''
    Convert COCO annotations to segmentation masks in class directories for prototype initialization.

    Args:
        annotation_file (str): Path to the COCO annotation file.
        image_directory (str): Path to the directory containing the images.
        save_path (str): Path to the directory where the segmentation masks will be saved.
    '''
    # Create the output directory if it doesn't exist
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # Load the COCO annotation file
    coco = COCO(annotation_file)

    # Get the list of category IDs
    category_ids = coco.getCatIds()


    # Loop through each category
    for category_id in category_ids:
        # Get the category information
        category_info = coco.loadCats(category_id)[0]
        category_name = category_info['name']
        category_directory = os.path.join(save_path, category_name)

        # Create a directory for the category
        if not os.path.exists(category_directory):
            os.makedirs(category_directory)

        # Get the image IDs containing the selected category
        image_ids = coco.getImgIds(catIds=category_id)
        
        # Subsample images if subset_size is given
        if subset_size is not None and subset_size < len(image_ids):
            image_ids = random.sample(image_ids, subset_size)

        for image_id in image_ids:
            # Load the image and annotations
            image_info = coco.loadImgs(image_id)[0]
            image_filename = image_info['file_name']
            image = cv2.imread(os.path.join(image_directory, image_filename))
            annotations = coco.loadAnns(coco.getAnnIds(imgIds=image_id, catIds=category_id))

            # Create a mask for the selected category
            mask = np.zeros(image.shape[:2], dtype=np.uint8)
            for annotation in annotations:
                bbox = list(map(int, annotation['bbox']))
                x, y, w, h = bbox
                mask[y:y + h, x:x + w] = 255

            # Save the image and mask
            image_filename_without_ext = os.path.splitext(image_filename)[0]
            mask_filename = f"{image_filename_without_ext}.mask{os.path.splitext(image_filename)[1]}"
            cv2.imwrite(os.path.join(category_directory, image_filename), image)
            cv2.imwrite(os.path.join(category_directory, mask_filename), mask)

        print(f"Processed category: {category_name}")

    print("Processing complete.")

In [24]:
# For val
subset_size = 10
for M in [1, 2, 3, 4, 5]:
    annotation_file=f'/home/gridsan/manderson/ovdsat/data/dior/train_coco_finetune_val-{M}.json'
    image_directory='/home/gridsan/manderson/ovdsat/data/dior/JPEGImages'
    save_path=f'/home/gridsan/manderson/ovdsat/data/init_data/dior_val-{M}'
    coco_to_seg(annotation_file, image_directory, save_path, subset_size)

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
Processed category: groundtrackfield
Processed category: baseballfield
Processed category: bridge
Processed category: Expressway-toll-station
Processed category: vehicle
Processed category: airplane
Processed category: airport
Processed category: tenniscourt
Processed category: trainstation
Processed category: storagetank
Processed category: stadium
Processed category: windmill
Processed category: ship
Processed category: golffield
Processed category: overpass
Processed category: chimney
Processed category: dam
Processed category: basketballcourt
Processed category: harbor
Processed category: Expressway-Service-area
Processing complete.


In [6]:
# For test
subset_size = 100
for M in [1]:#, 2, 3, 4, 5]:
    annotation_file=f'/home/gridsan/manderson/ovdsat/data/dior/val_coco-{M}.json'
    image_directory='/home/gridsan/manderson/ovdsat/data/dior/JPEGImages'
    save_path=f'/home/gridsan/manderson/ovdsat/data/init_data/dior_test-{M}'
    coco_to_seg(annotation_file, image_directory, save_path, subset_size)

loading annotations into memory...
Done (t=0.48s)
creating index...
index created!
Processed category: groundtrackfield
Processed category: baseballfield
Processed category: bridge
Processed category: Expressway-toll-station
Processed category: vehicle
Processed category: airplane
Processed category: airport
Processed category: tenniscourt
Processed category: trainstation
Processed category: storagetank
Processed category: stadium
Processed category: windmill
Processed category: ship
Processed category: golffield
Processed category: overpass
Processed category: chimney
Processed category: dam
Processed category: basketballcourt
Processed category: harbor
Processed category: Expressway-Service-area
Processing complete.


## From annotations file (don't use)
* Do the same but load from a .json file (gives image path)
* Select a subset of N images (or else it will take too long)

In [9]:
import json
import random
from pathlib import Path
from PIL import Image
import numpy as np
import xml.etree.ElementTree as ET

def apply_rectangular_mask_and_crop(image: Image.Image, mask: Image.Image) -> Image.Image:
    """Crops the image to the rectangular mask bounding box only (no masking)."""
    mask_array = np.array(mask.convert("L"))
    ys, xs = np.where(mask_array > 0)

    if xs.size == 0 or ys.size == 0:
        return Image.new("RGB", (1, 1))  # Empty image if no mask

    x_min, x_max = xs.min(), xs.max()
    y_min, y_max = ys.min(), ys.max()
    return image.crop((x_min, y_min, x_max + 1, y_max + 1))

def load_annotations(xml_path: Path):
    """Parse a Pascal VOC-style XML file and return object entries (optional)."""
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        objects = []
        for obj in root.findall("object"):
            name = obj.find("name").text
            bbox = obj.find("bndbox")
            box = {
                "name": name,
                "bbox": (
                    int(bbox.find("xmin").text),
                    int(bbox.find("ymin").text),
                    int(bbox.find("xmax").text),
                    int(bbox.find("ymax").text),
                ),
            }
            objects.append(box)
        return objects
    except Exception as e:
        print(f"[!] Failed to parse {xml_path.name}: {e}")
        return []

def process_from_json(
    json_path: str,
    image_root: str,
    annotations_dir: str,
    output_dir: str,
    subset_size: int = 100
):
    image_root = Path(image_root)
    annotations_dir = Path(annotations_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    with open(json_path, "r") as f:
        data = json.load(f)
    
    images = data["images"]
    if subset_size > 0:
        images = random.sample(images, subset_size)

    for img_info in images:
        try:
            file_name = img_info["file_name"]
            folder = img_info.get("folder", "")
            image_path = image_root / folder / file_name
            mask_path = image_path.with_name(image_path.stem + ".mask.jpg")
            xml_path = annotations_dir / (image_path.stem + ".xml")

            if not mask_path.exists():
                print(f"[!] Mask not found: {mask_path}")
                continue

            image = Image.open(image_path).convert("RGB")
            mask = Image.open(mask_path).convert("L")
            masked_cropped = apply_rectangular_mask_and_crop(image, mask)

            # Optional: load annotations
            _ = load_annotations(xml_path)

            # Save with preserved folder structure
            output_file = output_path / folder / file_name
            output_file.parent.mkdir(parents=True, exist_ok=True)
            masked_cropped.save(output_file)
            print(f"[✓] Saved: {output_file}")

        except Exception as e:
            print(f"[✗] Error processing {img_info}: {e}")


In [10]:
M=1

process_from_json(
    json_path=f"/home/gridsan/manderson/ovdsat/data/dior/train_coco_finetune_val-{M}.json",
    image_root="/home/gridsan/manderson/ovdsat/data/dior/JPEGImages",
    annotations_dir="/home/gridsan/manderson/ovdsat/data/dior/Annotations",
    output_dir=f"/home/gridsan/manderson/ovdsat/data/cropped_data/dior/val/dior-{M}",
    subset_size=10
)

[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/00499.mask.jpg
[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/01711.mask.jpg
[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/21022.mask.jpg
[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/17339.mask.jpg
[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/21084.mask.jpg
[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/19542.mask.jpg
[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/12238.mask.jpg
[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/02348.mask.jpg
[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/03136.mask.jpg
[!] Mask not found: /home/gridsan/manderson/ovdsat/data/dior/JPEGImages/17443.mask.jpg
